In [1]:
# Fusion ART Model for Professor Data using Word2Vec
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict
from collections import Counter

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk import ngrams
from nltk.corpus import stopwords

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Load a session
dill.load_session('notebook_env_wikipedia.db')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [4]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [134]:
# Function to search for n-grams for n up to 6 and return the n-grammed text.
# Functions accepts list of n-grams to be formed, the text and the common n-grams within the text, respectively.
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text


In [6]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_preprocessed = []
for entry in university:
    uni = entry.split(";")
    university_preprocessed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_processed to 1D lists for one hot encoding
university_1d = []
for universities in university_preprocessed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        

In [7]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_preprocessed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))
    

In [8]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [9]:
# Create a list of unique names, groups, universities and research interests to define the model schema
# (I refrained from using sets because it would randomise the one-hot ordering)
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass


In [83]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass



C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [164]:
# Remove common stop words from the retrieved Wikipedia articles
eng_stopwords = set(stopwords.words('english'))
for text in wiki_text:
    for word in text:
        if word in eng_stopwords:
            text.remove(word)

In [182]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in wiki_text:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)




In [183]:
# Create a list of common bigrams and trigrams from the Wikipedia articles
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))


In [184]:
# Search for research keyword n-grams from the Wikipedia articles and clean them
ngram_wiki_tokens = []

for text in wiki_text:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [186]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('it_text.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
            

# Remove stopwords in the custom text corpus
for word in testtext:
    if word in eng_stopwords:
        testtext.remove(word)
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [187]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in test_tokens:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)


In [188]:
# Create a list of common bigrams and trigrams from the custom corpus
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))

In [189]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []

for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [3]:
# Create a word2vec model that contains all research keywords
research_tokens = convert_to_2d(research_interest_1d, research_interest_per_person)
research_word2vec = Word2Vec(research_tokens, size = len(unique_research_interest), min_count = 0)


In [4]:
# Train the word2vec model from the cleaned, n-grammed Wikipedia tokens
research_word2vec.build_vocab(ngram_wiki_tokens, update = True)
research_word2vec.train(ngram_wiki_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(3722085, 4033045)

In [5]:
# Train the word2vec model by including the cleaned, n-grammed custom corpus tokens
research_word2vec.build_vocab(ngram_test_tokens, update = True)
research_word2vec.train(ngram_test_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(148098, 162805)

In [6]:
# Example call to find similar analogies (e.g. computer_graphics + computer - graphics)
research_word2vec.wv.most_similar(positive=['computer_graphics', 'computer'], negative=['graphics'])

[('algorithmthe', 0.8273654580116272),
 ('a_computer_network', 0.822250247001648),
 ('riskiest', 0.8215516805648804),
 ('computer_science', 0.8208965063095093),
 ('a_computer_vision', 0.8181934952735901),
 ('computer_network', 0.8136425018310547),
 ('floodingin', 0.8109545707702637),
 ('apollo', 0.8097070455551147),
 ('human_computer', 0.8087238073348999),
 ('asserted', 0.807906448841095)]

In [7]:
# Example call to retrieve vector of a word
research_word2vec["ad_hoc_and_mobile_networks"]

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 2.93206045e-04, -3.38491285e-04, -2.78101274e-04,  9.83046950e-04,
       -1.02769933e-03,  9.24153428e-04,  4.80568648e-04, -1.69386971e-03,
       -8.39546614e-04,  1.32930407e-03,  1.73121947e-03, -1.68091524e-03,
        1.24321296e-03,  1.85954303e-03, -2.00641254e-04, -1.17483537e-03,
       -9.16717749e-04, -1.04530749e-03, -7.04711420e-05, -1.26553094e-03,
       -1.22247892e-03,  5.00761205e-04,  8.04592797e-04, -7.58495706e-04,
       -1.41318026e-03, -4.74117784e-04,  1.68638281e-03, -1.69659988e-03,
       -1.52802400e-04, -4.40467775e-05, -1.22821913e-03, -1.08262035e-03,
        7.57855480e-04, -1.62379479e-03,  6.01276290e-04, -1.56382809e-03,
       -1.39317254e-03,  1.81871443e-03,  1.37814641e-04,  3.29818606e-04,
       -3.33928678e-04,  5.69199910e-04, -7.15767092e-04,  5.01246774e-04,
        2.50522455e-04,  1.46479683e-03, -1.20426332e-04, -4.35271912e-04,
       -1.12148223e-03, -3.56081815e-04,  1.73912616e-03, -2.79930653e-04,
        3.19660350e-04, -

In [8]:
# Calculate similarity score between two keywords
research_word2vec.wv.similarity("computer_graphics", "graphics")

0.92635626

In [9]:
# Find the top 5 most similar keywords for all research keywords
for keyword in sorted(unique_research_interest):
    print("Top 5 similar words from "+keyword+": ")
    print(research_word2vec.wv.most_similar(keyword, topn = 5))
    print()

Top 5 similar words from ad_hoc_and_mobile_networks: 
[('chipset', 0.16972897946834564), ('heckermans', 0.16884048283100128), ('reexamined', 0.15796972811222076), ('highschoollevel', 0.1567731350660324), ('transformerbased', 0.15451526641845703)]

Top 5 similar words from agent_oriented_software_engineering: 
[('software_engineering', 0.928935706615448), ('freelibre', 0.885480523109436), ('pseven', 0.8714703917503357), ('engineering', 0.8678944706916809), ('software_engineers', 0.8439077734947205)]

Top 5 similar words from algorithms: 
[('antialiasing', 0.9236392974853516), ('techniques', 0.9217177629470825), ('methods', 0.897507905960083), ('nbest', 0.8836230635643005), ('compression_algorithms', 0.8816787600517273)]

Top 5 similar words from algorithms_and_data_structure: 
[('bertsekas', 0.2069839984178543), ('columnist', 0.18753814697265625), ('entrepreneurship', 0.1874026209115982), ('counterfactual', 0.1799372136592865), ('coursera', 0.17927870154380798)]

Top 5 similar words fro

[('molecular', 0.9820254445075989), ('computational_biology', 0.9784550666809082), ('physics', 0.9778605699539185), ('mathematical_modeling_computational', 0.9749706387519836), ('biochemistry', 0.973071277141571)]

Top 5 similar words from computational_intelligence: 
[('endowed', 0.947769045829773), ('artificial_general_intelligence', 0.9339401125907898), ('mad', 0.92060786485672), ('intelligencethe', 0.9174039959907532), ('bladders', 0.9096550345420837)]

Top 5 similar words from computational_social_choice: 
[('policymaking', 0.9825296998023987), ('cooperation', 0.9802403450012207), ('obtains', 0.9775935411453247), ('students’', 0.9771437048912048), ('reactions', 0.9769716262817383)]

Top 5 similar words from computational_social_science: 
[('egocentric', 0.9682998657226562), ('vernor', 0.9621878862380981), ('bachelor', 0.9615944027900696), ('computer_science_aims', 0.9594775438308716), ('biomaterials', 0.9584701657295227)]

Top 5 similar words from computational_systems_biology: 
[

[('advancements', 0.9769251346588135), ('emerging', 0.9757338762283325), ('origins', 0.9734771847724915), ('development_emerging', 0.9731960892677307), ('geoinformatics', 0.9717893600463867)]

Top 5 similar words from enterprise_architecture: 
[('standardization', 0.9873228073120117), ('procurement', 0.9845641851425171), ('center_design', 0.9834672808647156), ('comprises_disciplines_related', 0.9810741543769836), ('governmental', 0.9797008037567139)]

Top 5 similar words from enterprise_blockchain_applications: 
[('smarterthanhuman', 0.21640755236148834), ('cliodynamics', 0.2111150175333023), ('‘‘', 0.21023954451084137), ('fermentation', 0.20967216789722443), ('syllabi', 0.20399653911590576)]

Top 5 similar words from enterprise_mobility_platform: 
[('collaborative_internet_computing_technologies_and_applications', 0.20417293906211853), ('many_core_systems', 0.19991737604141235), ('nonexhaustive', 0.19347141683101654), ('pornographic', 0.16988734900951385), ('slowdown', 0.1685134321451

[('premium', 0.7830092310905457), ('siteswith', 0.7342382073402405), ('lsp', 0.7277327179908752), ('internet', 0.7158572673797607), ('selfmonitoring', 0.7137290835380554)]

Top 5 similar words from low_power_reconfigurable_computing: 
[('confounding', 0.18047703802585602), ('avariety', 0.17677141726016998), ('spatialsemantic', 0.1707259714603424), ('dcc', 0.16692417860031128), ('plowing', 0.16569991409778595)]

Top 5 similar words from machine_learning: 
[('finetuning', 0.9631698727607727), ('transfer_learning', 0.9594578146934509), ('tool_monitoring', 0.9521664381027222), ('reinforcement_learning', 0.950230062007904), ('the_federated_learning', 0.9493115544319153)]

Top 5 similar words from machine_learning_and_knowledge_discovery: 
[('georeferenced', 0.2182236909866333), ('cropping', 0.21656136214733124), ('propensity', 0.2121867835521698), ('such', 0.20437060296535492), ('unfeasible', 0.2022877037525177)]

Top 5 similar words from many_core_systems: 
[('exerting', 0.5601268410682678

[('telephoneswitching', 0.9597154855728149), ('wireless_networks', 0.9586228728294373), ('adhoc', 0.9409193396568298), ('local_area_networks', 0.9391568303108215), ('selfhealing', 0.9339436888694763)]

Top 5 similar words from parallel_algorithms_and_architectures: 
[('intelligent_multi_agents', 0.20051224529743195), ('biometric_image_processing', 0.17399469017982483), ('enterprise_mobility_platform', 0.16737577319145203), ('pursuance', 0.1567252278327942), ('visualization_on_the_grid', 0.1564294546842575)]

Top 5 similar words from parallel_and_distributed_simulation: 
[('ggg', 0.18049025535583496), ('gatekeeper', 0.17552730441093445), ('payg', 0.17182554304599762), ('minedetecting', 0.1671532839536667), ('ipados', 0.16389164328575134)]

Top 5 similar words from pattern_recognition: 
[('pattern', 0.9817470908164978), ('speech_recognition', 0.9426707029342651), ('recognition', 0.9354987740516663), ('automatic_speech_recognition', 0.9266421794891357), ('statistical', 0.9188041687011719)

[('dxygeq', 0.18939204514026642), ('anonymization', 0.17526783049106598), ('lawton', 0.16753622889518738), ('rivest–shamir–adleman', 0.15651944279670715), ('honesty', 0.15090833604335785)]

Top 5 similar words from synthesis_for_emerging_technology: 
[('boxoffice', 0.1616712361574173), ('unapologetic', 0.15464627742767334), ('doublespending', 0.14994817972183228), ('reidentification', 0.1444898545742035), ('something_user', 0.14248228073120117)]

Top 5 similar words from system_level_design: 
[('twogiro', 0.8877086639404297), ('videowindow', 0.8813989758491516), ('the_design_the', 0.8732262849807739), ('orientais', 0.8716607093811035), ('decimal', 0.8689892292022705)]

Top 5 similar words from system_performance_analysis: 
[('chipit', 0.2153167873620987), ('calc', 0.19670958817005157), ('humanmade', 0.1958881914615631), ('overloads', 0.1936415284872055), ('semiautonomy', 0.18721948564052582)]

Top 5 similar words from text_mining: 
[('term_data_mining', 0.9749606847763062), ('web_usage

In [12]:
# One Hot Encoding for name, group and university
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)


In [13]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()



In [14]:
# Provide weight vectors for each research interest
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_word2vec[res_int])

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [15]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_weights, research_interest_per_person)

In [16]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []
for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])
    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)])

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [17]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))


In [18]:
# Example 2D research interest vector. This is the research keyword vector for Alexei Sourin
print(normalised_research_interest_2d[0])

[0.7502326381467871, 0.17045754885698766, 0.6648618891314976, 0.7286885213815398, 0.6279767674335179, 0.40420036308075563, 0.44213849501760016, 0.41796893357945775, 0.2125228833884819, 0.3276878007499624, 0.6515365576163532, 0.38717219997565416, 0.5177317039702222, 0.7456806522801664, 0.4810354506649106, 0.467473715295032, 0.6171803706008244, 0.31440244555325464, 0.31081766339756683, 0.3460090509799768, 0.5035748864978885, 0.5889029057871761, 0.8431558968814281, 0.39896933409344343, 0.3407439050465997, 0.7690076578088565, 0.5879181867020244, 0.4209003485703699, 0.5435123774095807, 0.5612612247716222, 0.8049318794518617, 0.8510390910554343, 0.2663271700265207, 0.5682915553063199, 0.6010979652323147, 0.7962432420837253, 0.452709858744945, 0.5816386200659746, 0.6653515472382213, 0.5000147466323278, 0.4771893622797161, 0.7083713577410625, 0.3617377116980258, 0.630485442539818, 0.34291012762042594, 0.4373220066629937, 0.26967040378090407, 0.976677752936088, 0.4615298657826051, 0.74779952274

In [19]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [20]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [21]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

In [22]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file, noise_limit):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")

    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        query_research[research_index] = research_word2vec[query_research[research_index]].tolist()


    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1
        
    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    normalised_research_vector = normalise_vector(research_vector)
    

    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            normalised_research_vector[index] = 1 - normalised_research_vector[index]
            
    # Define the complement-coded research vector
    compl_vector = []
    for bit in normalised_research_vector:
        compl_vector.append(1 - bit)
    

    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector, 'vcompl': compl_vector}], refresh = False)
    # model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector}])
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [23]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [24]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [25]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.2)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [26]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.3)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [27]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.4)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [28]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.5)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [29]:
# Research keywords query for 60% noise
output_file = open("output_sixty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.6)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [30]:
# Research keywords query for 70% noise
output_file = open("output_seventy_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.7)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [31]:
# Research keywords query for 80% noise
output_file = open("output_eighty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.8)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [32]:
# Research keywords query for 90% noise
output_file = open("output_ninety_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.9)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [33]:
# Research keywords query for 100% noise
output_file = open("output_hundred_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
